# Лабораторная работа №6
## Задание:
1 . Провести классификацию найденного датасета, методами CatBoost . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

## Лабораторная работа:
### Подключение библиотек


In [ ]:
!pip install catboost

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from catboost import CatBoostClassifier


### Загружаю данные и вывод датасет


In [ ]:
Dataset = pd.read_csv('survey.csv')

### Создание искомого столбца

In [ ]:
label_encoder = LabelEncoder()
Dataset['CompTotal Encoded'] = label_encoder.fit_transform(Dataset['CompTotal'])

### Очистка данных

In [ ]:
non_numeric_columns = Dataset.select_dtypes(exclude=['number']).columns.tolist()
Dataset = Dataset.drop(columns=non_numeric_columns)

# Заполнение пустых значений средними значениями
columns_to_fill = Dataset.select_dtypes(include=['number']).columns.tolist()
for column in columns_to_fill:
    mean_value = Dataset[column].mean()
    Dataset[column].fillna(mean_value, inplace=True)

### Поиск лучших метрик и лучших количества соседей

In [ ]:
X = Dataset.drop(columns=['CompTotal Encoded'])
y = Dataset['CompTotal Encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Параметры для настройки CatBoost
catboost_params = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8]
}

# Создание модели CatBoost
catboost_model = CatBoostClassifier(verbose=False)

# Подбор лучших параметров с помощью GridSearchCV
catboost_grid_search = GridSearchCV(catboost_model, catboost_params, cv=5)
catboost_grid_search.fit(X_train, y_train)

# Лучшие параметры
best_catboost_params = catboost_grid_search.best_params_
print(f"Лучшие параметры для CatBoost: {best_catboost_params}")


Лучшие параметры для CatBoost: {'depth': 4, 'iterations': 200, 'learning_rate': 0.05}


#### Обучение на лучших метриках и количестве соседей

In [ ]:
# Переобучение модели с лучшими параметрами на всем тренировочном наборе данных
best_catboost_model = CatBoostClassifier(**best_catboost_params, verbose=False)
best_catboost_model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred_catboost = best_catboost_model.predict(X_test)

# Оценка точности
accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
print(f"Точность CatBoost на тестовых данных: {accuracy_catboost}")

# Отчет о классификации для CatBoost
print("Classification Report для CatBoost:")
print(classification_report(y_test, y_pred_catboost))

Точность CatBoost на тестовых данных: 0.32875
Classification Report для CatBoost:
              precision    recall  f1-score   support

           0       0.33      0.40      0.36      5306
           1       0.34      0.22      0.26      5416
           2       0.33      0.37      0.35      5278

    accuracy                           0.33     16000
   macro avg       0.33      0.33      0.32     16000
weighted avg       0.33      0.33      0.32     16000

